# NOAA Atlas 14 Download

## User Inputs

In [5]:
latitude = 57.959019
longitude = -136.224579
folder_path = r"C:\Users\gabe.benitez\Documents\test"  # Change to your desired folder

### Run Code (DON'T MODIFY)

In [6]:
# DON'T MODIFY
import base64
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

def generate_pdf(latitude, longitude, folder_path, estimate_type, filename_suffix):
    file_name = f"{latitude}_{longitude}_{filename_suffix}.pdf"
    pdf_path = os.path.join(folder_path, file_name)

    driver = webdriver.Chrome()
    driver.get("https://hdsc.nws.noaa.gov/pfds/pfds_map_ak.html")
    wait = WebDriverWait(driver, 30)

    # If estimate_type is 'intensity', select it from dropdown
    if estimate_type == "intensity":
        wait.until(EC.presence_of_element_located((By.XPATH, "//select[@id='type']/option")))
        type_dropdown = driver.find_element(By.ID, "type")
        Select(type_dropdown).select_by_value("intensity")

    # Enter lat/lon
    lat_input = wait.until(EC.presence_of_element_located((By.ID, "manLat")))
    lon_input = wait.until(EC.presence_of_element_located((By.ID, "manLon")))
    lat_input.clear()
    lat_input.send_keys(str(latitude))
    lon_input.clear()
    lon_input.send_keys(str(longitude))

    # Submit
    driver.find_element(By.ID, "latlonButton").click()

    # Wait for Print Page button and click
    print_btn = wait.until(EC.element_to_be_clickable((By.ID, "prtPg")))
    print_btn.click()

    # Switch to new tab
    wait.until(lambda d: len(d.window_handles) > 1)
    driver.switch_to.window(driver.window_handles[-1])

    # Wait for rendering
    time.sleep(20)

    # Save PDF
    pdf = driver.execute_cdp_cmd("Page.printToPDF", {"printBackground": True})
    with open(pdf_path, "wb") as f:
        f.write(base64.b64decode(pdf['data']))

    driver.quit()
    print(f"✅ PDF saved to: {pdf_path}")

# === Run both types ===
generate_pdf(latitude, longitude, folder_path, estimate_type="depth", filename_suffix="Precipitation_Depth_Frequency_Estimates")
generate_pdf(latitude, longitude, folder_path, estimate_type="intensity", filename_suffix="Precipitation_Intensity_Frequency_Estimates")


✅ PDF saved to: C:\Users\gabe.benitez\Documents\test\57.959019_-136.224579_Precipitation_Depth_Frequency_Estimates.pdf
✅ PDF saved to: C:\Users\gabe.benitez\Documents\test\57.959019_-136.224579_Precipitation_Intensity_Frequency_Estimates.pdf
